In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf

In [2]:
data= pd.read_csv('../input/titanic/train.csv')

In [3]:
print(data.shape)
data.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
data = data[['Survived', 'Pclass', 'Sex', 'Age', 'Fare']]

In [5]:
data = data.dropna()
print(data.shape)

(714, 5)


In [6]:
target = data.pop('Survived')

In [7]:
categorical_feature_names = ['Pclass','Sex']
numeric_feature_names = ['Fare', 'Age']
predicted_feature_name = ['Survived']

In [8]:
def create_tensor_dict(data, categorical_feature_names):
    inputs = {}
    for name, column in data.items():
      if type(column[0]) == str:
        dtype = tf.string
      elif (name in categorical_feature_names):
        dtype = tf.int64
      else:
        dtype = tf.float32

      inputs[name] = tf.keras.Input(shape=(), name=name, dtype=dtype)
    return inputs

inputs = create_tensor_dict(data, categorical_feature_names)


In [9]:
def stack_dict(inputs, fun=tf.stack):
    values = []
    for key in sorted(inputs.keys()):
      values.append(tf.cast(inputs[key], tf.float32))

    return fun(values, axis=-1)

In [10]:
def create_normalizer(numeric_feature_names, data):
    numeric_features = data[numeric_feature_names]
    
    normalizer = tf.keras.layers.Normalization(axis=-1)
    normalizer.adapt(stack_dict(dict(numeric_features)))  
    return normalizer

In [11]:
def normalize_numeric_input(numeric_feature_names, inputs, normalizer):
    numeric_inputs = {}
    for name in numeric_feature_names:
      numeric_inputs[name]=inputs[name]

    numeric_inputs = stack_dict(numeric_inputs)
    numeric_normalized = normalizer(numeric_inputs) 
    return numeric_normalized

In [12]:
normalizer = create_normalizer(numeric_feature_names, data)
numeric_normalized = normalize_numeric_input(numeric_feature_names, inputs, normalizer)
print(numeric_normalized)

2022-07-03 23:35:05.731584: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-07-03 23:35:05.846150: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


KerasTensor(type_spec=TensorSpec(shape=(None, 2), dtype=tf.float32, name=None), name='normalization/truediv:0', description="created by layer 'normalization'")


In [13]:
preprocessed = []
preprocessed.append(numeric_normalized)

In [14]:
def one_hot_encode_categorical_features(categorical_feature_names, data, inputs):
    one_hot = []
    for name in categorical_feature_names:
      value = sorted(set(data[name]))

      if type(value[0]) is str:
        lookup = tf.keras.layers.StringLookup(vocabulary=value, output_mode='one_hot')
      else:
        lookup = tf.keras.layers.IntegerLookup(vocabulary=value, output_mode='one_hot')

      x = inputs[name][:, tf.newaxis]
      x = lookup(x)
      one_hot.append(x)
    return one_hot

In [15]:
one_hot = one_hot_encode_categorical_features(categorical_feature_names, data, inputs)
preprocessed = preprocessed + one_hot


In [16]:
preprocesssed_result = tf.concat(preprocessed, axis=-1)


In [17]:
preprocessor = tf.keras.Model(inputs, preprocesssed_result)

In [18]:
preprocessor(dict(data.iloc[:1]))

<tf.Tensor: shape=(1, 9), dtype=float32, numpy=
array([[-0.5303766, -0.5189777,  0.       ,  0.       ,  0.       ,
         1.       ,  0.       ,  0.       ,  1.       ]], dtype=float32)>

In [19]:
network = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation='relu'),
  tf.keras.layers.Dense(10, activation='relu'),
  tf.keras.layers.Dense(1)
])

In [20]:
x = preprocessor(inputs)
result = network(x)
model = tf.keras.Model(inputs, result)

In [21]:
model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])

In [22]:
history = model.fit(dict(data), target, epochs=25, batch_size=8)

Epoch 1/25
90/90 [==============================] - 1s 2ms/step - loss: 0.6630 - accuracy: 0.6064
Epoch 2/25
90/90 [==============================] - 0s 2ms/step - loss: 0.6056 - accuracy: 0.6611
Epoch 3/25
90/90 [==============================] - 0s 2ms/step - loss: 0.5350 - accuracy: 0.7213
Epoch 4/25
90/90 [==============================] - 0s 2ms/step - loss: 0.4827 - accuracy: 0.7871
Epoch 5/25
90/90 [==============================] - 0s 2ms/step - loss: 0.4574 - accuracy: 0.7913
Epoch 6/25
90/90 [==============================] - 0s 2ms/step - loss: 0.4464 - accuracy: 0.7927
Epoch 7/25
90/90 [==============================] - 0s 2ms/step - loss: 0.4386 - accuracy: 0.7955
Epoch 8/25
90/90 [==============================] - 0s 3ms/step - loss: 0.4339 - accuracy: 0.7969
Epoch 9/25
90/90 [==============================] - 0s 3ms/step - loss: 0.4300 - accuracy: 0.7983
Epoch 10/25
90/90 [==============================] - 0s 3ms/step - loss: 0.4272 - accuracy: 0.7997
Epoch 11/25
90/90 [

In [23]:
from sklearn.model_selection import train_test_split

train_data, val_data, train_target, val_target = train_test_split(data,target, train_size=0.8)
history = model.fit(dict(train_data), train_target, validation_data=(dict(val_data), val_target), epochs=20, batch_size=8)

Epoch 1/20
72/72 [==============================] - 1s 8ms/step - loss: 0.4297 - accuracy: 0.8126 - val_loss: 0.3293 - val_accuracy: 0.8601
Epoch 2/20
72/72 [==============================] - 0s 4ms/step - loss: 0.4298 - accuracy: 0.8126 - val_loss: 0.3338 - val_accuracy: 0.8601
Epoch 3/20
72/72 [==============================] - 0s 4ms/step - loss: 0.4285 - accuracy: 0.8109 - val_loss: 0.3333 - val_accuracy: 0.8601
Epoch 4/20
72/72 [==============================] - 0s 5ms/step - loss: 0.4279 - accuracy: 0.8144 - val_loss: 0.3326 - val_accuracy: 0.8601
Epoch 5/20
72/72 [==============================] - 0s 4ms/step - loss: 0.4274 - accuracy: 0.8074 - val_loss: 0.3332 - val_accuracy: 0.8601
Epoch 6/20
72/72 [==============================] - 0s 3ms/step - loss: 0.4262 - accuracy: 0.8109 - val_loss: 0.3367 - val_accuracy: 0.8601
Epoch 7/20
72/72 [==============================] - 0s 3ms/step - loss: 0.4265 - accuracy: 0.8091 - val_loss: 0.3346 - val_accuracy: 0.8601
Epoch 8/20
72/72 [==

In [24]:
results = model.evaluate(dict(train_data), train_target, batch_size=128)
print("test loss, test accuracy:", results)


5/5 [==============================] - 0s 3ms/step - loss: 0.4191 - accuracy: 0.8091
test loss, test accuracy: [0.4191390573978424, 0.8091068267822266]
